In [588]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline

from xgboost import XGBRegressor, plot_importance
from lightgbm import LGBMRegressor
import shap

In [589]:
import os
os.chdir(r"C:\Users\lizzy\OneDrive\바탕 화면\실전프로젝트")
print(os.getcwd())  # 잘 바뀌었는지 확인


C:\Users\lizzy\OneDrive\바탕 화면\실전프로젝트


In [590]:
df_milk = pd.read_csv("regression_dataset.csv")

In [591]:
df_milk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58441 entries, 0 to 58440
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   누적착유일(연계)     58441 non-null  int64  
 1   전산차비유지속성      58440 non-null  float64
 2   현재산차비유지속성     58440 non-null  float64
 3   유량            58441 non-null  int64  
 4   건유전마지막유량_log  33724 non-null  float64
 5   전산차건유전유량      33724 non-null  float64
 6   산차            54992 non-null  float64
 7   농후사료급여량(연계)   58441 non-null  float64
 8   공태일수_log      58441 non-null  float64
 9   출생연도          58441 non-null  int64  
 10  출생월           58441 non-null  int64  
 11  출생일           58441 non-null  int64  
dtypes: float64(7), int64(5)
memory usage: 5.4 MB


In [592]:
# 출생연도가 이미 숫자형(int)이라고 가정
# 만약 object라면 df['출생연도'] = df['출생연도'].astype(int) 필요

# train: 출생연도 == 2020
train = df_milk[df_milk['출생연도'] == 2020]

# test: 출생연도 == 2021
test = df_milk[df_milk['출생연도'] == 2021]

# 독립변수(X), 종속변수(y) 분리 예시
X_train = train.drop(columns=['유량', '출생연도'])
y_train = train['유량']

X_test = test.drop(columns=['유량', '출생연도'])
y_test = test['유량']


In [593]:
X_train

,누적착유일(연계),전산차비유지속성,현재산차비유지속성,건유전마지막유량_log,전산차건유전유량,산차,농후사료급여량(연계),공태일수_log,출생월,출생일
57435,16,0.0,0.0,NaN,NaN,1.0,0.0,0.0,1,13
57436,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17
57437,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17
57438,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17
57439,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17
57440,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17
57441,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17
57442,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17
57443,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17
57444,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1,17


In [594]:
y_train

57435    76149919
57436    76292379
57437    76292518
57438    76292657
57439    76292799
57440    76292943
57441    76293082
57442    76293207
57443    76293316
57444    76293421
57445    76293519
57446    76293620
57447    76293723
57448    76293826
57449    76293930
57450    76294033
57451    76294136
57452    76294243
57453    76294352
57454    76294460
Name: 유량, dtype: int64